In [4]:
import scrapy
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys
sys.path.append("../..")
from case_scrapy.items import case_Items
from selenium.webdriver.support.ui import Select
from scrapy import Selector
import json
import re
import time
import datetime
import lxml.html as lh

# Scrape Links

In [20]:
#chromedriver_path = "/Users/yuchen.zhang/Downloads/Installations/Chromedriver"
#driver = webdriver.Chrome(executable_path = chromedriver_path)

In [21]:
case_links = []
i = 1
first_page_html = requests.get("http://securities.stanford.edu/filings.html?page=1").text
next_page_html = requests.get("http://securities.stanford.edu/filings.html?page=" + str(i+1)).text

while first_page_html != next_page_html:
    try:
        driver.get("http://securities.stanford.edu/filings.html?page=" + str(i))
        sel_case = Selector(text=driver.page_source)
        case_links_raw = sel_case.css('tr:nth-child(n)::attr(onclick)').getall()
        case_links.append(case_links_raw)
        
        i += 1
        next_page_html = requests.get("http://securities.stanford.edu/filings.html?page=" + str(i)).text
    except: break

total_pages = i

In [22]:
# Change the links to actual links
case_links_clean = []
for i in range(0, len(case_links)):
    for j in range(0, len(case_links[i])):
        case_links_clean.append("http://securities.stanford.edu/" + case_links[i][j].replace("window.location=","").replace("'",""))

In [56]:
#pd.DataFrame(case_links_clean).to_excel("case_links_all.xlsx")

# Scrape a whole table

In [394]:
page = requests.get("http://securities.stanford.edu/filings.html?page=1")
soup = BeautifulSoup(page.content, 'lxml')

In [395]:
print(soup.title.text)

Securities Class Action Clearinghouse: Filings Database


In [396]:
# Get headers of the table
case_table = soup.find("table", attrs={"class": "table table-bordered table-striped table-hover"})
table_header = soup.find('thead')
header = table_header.find_all('th')
header = [x.text.strip() for x in header]
print(header)

['Filing Name', 'Filing Date', 'District Court', 'Exchange', 'Ticker']


In [397]:
# Get table of the body, saving element row by row
table_body = soup.find('tbody')
rows = table_body.find_all('tr')
table_list = []
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    table_list.append(cols)

In [398]:
# Save the complete table
summary_table = pd.DataFrame(table_list)
summary_table.columns = header

In [198]:
# Make a function 
def get_summary_table(url):
    # Get page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get headers of the table
    case_table = soup.find("table", attrs={"class": "table table-bordered table-striped table-hover"})
    table_header = soup.find('thead')
    header = table_header.find_all('th')
    header = [x.text.strip() for x in header]
    
    # Get table of the body, saving element row by row
    table_body = soup.find('tbody')
    rows = table_body.find_all('tr')
    table_list = []
    for row in rows:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        table_list.append(cols)
        
    # Save the complete table
    summary_table = pd.DataFrame(table_list)
    summary_table.columns = header
    
    return summary_table

In [238]:
table_all = pd.DataFrame()
for i in range(1, total_pages):
    table_all = table_all.append(get_summary_table("http://securities.stanford.edu/filings.html?page=" + str(i)))

In [240]:
#pd.DataFrame(table_all).to_excel("summary_table_all.xlsx")

# Scrape each page

In [5]:
df_case_links_clean = pd.read_excel("../../../data/case_links_all.xlsx")
case_links_clean = df_case_links_clean[0].tolist()

In [135]:
#temp = case_links_clean[0]
temp = "http://securities.stanford.edu/filings-case.html?id=101046"
case_page = requests.get(temp)
case_soup = BeautifulSoup(case_page.text, 'lxml')

In [136]:
# Save the final results in a dictionary
del c
c = case_Items()

In [137]:
# Basic Summary
case_url = temp

summary_section = case_soup.find(id = "summary")
case_name = summary_section.find("div", attrs = {"class" : "page-header hidden-phone"}).text.strip().replace('Case Summary\n','')

c["case_name"] = case_name
c["url"] = case_url
case_name

'Computron Software, Inc. Securities Litigation'

In [138]:
c["url"], c["case_name"]

('http://securities.stanford.edu/filings-case.html?id=101046',
 'Computron Software, Inc. Securities Litigation')

In [139]:
# Case Summary
case_section = case_soup.find(id = "summary")
case_para = [x.text.strip() for x in case_section.find_all("div", attrs = {"class":"span12"})]
c["case_brief"] = ''.join(case_para)

case_status = case_section.find("p").getText().replace("\n","").replace("\xa0","").replace("\t","")
case_status_list = [["Case Breif"] + case_para]
for i, x in enumerate(case_status.split("  ")):
    x = x.strip()
    if x == "Case Status:":
        c["case_status"] = case_status.split("  ")[1]
    else:
        if x[:12] == "On or around":
            c["date_of_last_review"] = x[13:(13+10)]

In [140]:
c["case_status"],c["date_of_last_review"], c["case_brief"]

('SETTLED',
 '03/11/1998',
 'According to the Company’s 10-K For The Fiscal Year Ended December 31, 1997, on March 6, 1998 the District Court issued a final order approving a settlement in the class action securities litigation, IN RE COMPUTRON SOFTWARE, INC. SECURITIES LITIGATION, Master File No. 96-1911 (AJL), brought against the Company and certain of its present and former officers and directors in the United States District Court for the District of New Jersey.In a press release dated October 28, 1997, Computron Software, Inc., announced today, that subject to final documentation and approval of the court, a settlement has been reached with lead counsel representing the certified class in the class action securities litigation pending against the company and certain of its present and former officers and directors in the United States District Court of New Jersey.  As its share of the settlement, the company will pay $1 million in cash, and an additional $5 million, which may be p

In [141]:
# Plaintiff
plaintiff_section = case_soup.find_all("ol", class_="styled")
plaintiff = [x.text.strip() for x in plaintiff_section]
plaintiff_list = plaintiff[0].split('\n\n\n')

pstr = ""
for x in plaintiff_list:
    if pstr == "":
        pstr += x
    else:
        pstr += ";" + x
pstr

c["plaintiffs"] = pstr

In [143]:
c["plaintiffs"]

'Bernstein Litowitz Berger & Grossmann LLP (Westfield, NJ);Goodkind Labaton Rudoff & Sucharow LLP'

In [142]:
# Company
company_section = case_soup.find(id = "company")
company = [x.text.strip() for x in company_section.find_all("div", attrs = {"class":"span4"})]

for x in company:
    if len(x.split(": "))==2:
        key,value = x.split(": ")
    else:
        key,value = (x + " ").split(": ")
    key = '_'.join(key.split(" ")).lower()
    c[key] = value

In [144]:
c

{'case_brief': 'According to the Company’s 10-K For The Fiscal Year Ended '
               'December 31, 1997, on March 6, 1998 the District Court issued '
               'a final order approving a settlement in the class action '
               'securities litigation, IN RE COMPUTRON SOFTWARE, INC. '
               'SECURITIES LITIGATION, Master File No. 96-1911 (AJL), brought '
               'against the Company and certain of its present and former '
               'officers and directors in the United States District Court for '
               'the District of New Jersey.In a press release dated October '
               '28, 1997, Computron Software, Inc., announced today, that '
               'subject to final documentation and approval of the court, a '
               'settlement has been reached with lead counsel representing the '
               'certified class in the class action securities litigation '
               'pending against the company and certain of its present 

In [145]:
# fic (first identified complaint)
fic_section = case_soup.find(id = "fic")
fic = [x.text.strip() for x in fic_section.find_all("div", attrs = {"class":"span4"})]

for x in fic:
    if len(x.split(": "))==2:
        key,value = x.split(": ")
    else:
        key,value = (x + " ").split(": ")
    key = re.sub('[^a-zA-Z0-9\n\.]', ' ',key).strip()
    key = '_'.join(key.split(" ")).lower()
    c[key] = value 

In [146]:
c

{'case_brief': 'According to the Company’s 10-K For The Fiscal Year Ended '
               'December 31, 1997, on March 6, 1998 the District Court issued '
               'a final order approving a settlement in the class action '
               'securities litigation, IN RE COMPUTRON SOFTWARE, INC. '
               'SECURITIES LITIGATION, Master File No. 96-1911 (AJL), brought '
               'against the Company and certain of its present and former '
               'officers and directors in the United States District Court for '
               'the District of New Jersey.In a press release dated October '
               '28, 1997, Computron Software, Inc., announced today, that '
               'subject to final documentation and approval of the court, a '
               'settlement has been reached with lead counsel representing the '
               'certified class in the class action securities litigation '
               'pending against the company and certain of its present 

In [147]:
# fic documents
fic_documents = case_soup.find("table", {"class", "table table-bordered table-striped table-hover"})

# get titles
fic_table_header = fic_documents.find('thead')
fic_header = fic_documents.find_all('th')
fic_header = [x.text.strip() for x in fic_header]

# get body
fic_table_body = case_soup.find('tbody')
fic_rows = fic_table_body.find_all('tr')
fic_table_list = []
for row in fic_rows:
    fic_cols=row.find_all('td')
    fic_cols=[x.text.strip() for x in fic_cols]
    fic_table_list.append(fic_cols)
    
# make a table
fic_summary_table = pd.DataFrame(fic_table_list)
try: 
    fic_summary_table.columns = fic_header
except:pass

In [148]:
# fic documents pdf
fic_links = fic_documents.find_all("tr", {"onclick" : re.compile("^window.location")})

fic_links_list = []
for x in fic_links:
    x = "http://securities.stanford.edu/" + x.get("onclick").replace("window.location=","").replace("'","")
    fic_links_list.append(x)

try:
    fic_summary_table['link'] = pd.DataFrame(fic_links_list)
except:pass

c["fic_links_list"] = fic_links_list
c["fic_summary_table"] = fic_summary_table.to_json()


In [149]:
c["fic_links_list"]

[]

In [150]:
c["fic_summary_table"]

'{}'

In [134]:
c

{'case_brief': "According to the Company's  FORM 10-Q for the quarterly period "
               'ended September 30, 1999, on October 28, 1999, the federal '
               'district court approved a settlement to the class action '
               'lawsuits against ValuJet Airlines. Although the Company denied '
               'that it violated any of its obligations under the federal '
               'securities laws, it agreed to pay $2.5 million in cash and '
               '$2.5 million in common stock in the settlement.  In 1996, '
               'several stockholder class action suits were filed against the '
               "Company andcertain of its executive officers ('Defendants').  "
               'The consolidated lawsuitssought class certification for all '
               'purchasers of stock in the Company during periods beginning on '
               'or after June 1995 and ending on or before June 18, 1996, and '
               'are based on allegedly misleading public s

In [151]:
c.keys()

dict_keys(['case_name', 'url', 'case_brief', 'case_status', 'date_of_last_review', 'plaintiffs', 'sector', 'industry', 'headquarters', 'ticker_symbol', 'company_market', 'market_status', 'court', 'docket', 'judge', 'date_filed', 'class_period_start', 'class_period_end', 'fic_links_list', 'fic_summary_table'])

# Bulk download pdfs

In [264]:
import os
import requests
from time import time
from multiprocessing.pool import ThreadPool

In [307]:
folder = "/Users/yuchen.zhang/Documents/Projects/Class_Action/class_action_git/data/Case Filings"

In [308]:
def url_response(url):
    path, url = url
    r = requests.get(url, stream = True)
    
    with open(folder + "/" + path,"wb") as pdf: 
        for chunk in r.iter_content(chunk_size=1024): 

             # writing one chunk at a time to pdf file 
             if chunk: 
                 pdf.write(chunk)

In [356]:
case_dict["fic_summary_table"].iloc[0]

No                                                                1
Document Title                               Class Action Complaint
Filing Date                                              07/07/2020
link              http://securities.stanford.edu/filings-documen...
Name: 0, dtype: object

In [323]:
for i, j in case_dict["fic_summary_table"].iterrows():
    filename = case_dict["DOCKET #"] + "_" + case_dict["fic_summary_table"]["Document Title"][i] + "_" + case_dict["fic_summary_table"]["Filing Date"][i].replace("/","")
    print(filename)
    url_response([filename,
                  case_dict["fic_summary_table"]["link"][i]])

20-CV-81063_Class Action Complaint_07072020
20-CV-81063_U.S. District Court Civil Docket_07072020
